In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
import numpy as np

def f1(x: np.ndarray, t: np.ndarray) -> np.ndarray:
    return 1.0 / np.cosh(x + 3.0) * np.exp(2.3J * t)

def f2(x: np.ndarray, t: np.ndarray) -> np.ndarray:
    return 2.0 / np.cosh(x) * np.tanh(x) * np.exp(2.8J * t)

x = np.linspace(-5.0, 5.0, 2 ** 10 + 1)
t = np.linspace(0.0, 4.0 * np.pi, 2 ** 8 + 1)

xgrid, tgrid = np.meshgrid(x, t)

X1 = f1(xgrid, tgrid)
X2 = f2(xgrid, tgrid)
X = X1 + X2

In [ ]:
from matplotlib import pyplot as plt

titles = ['$f_1(x,t)$', '$f_2(x,t)$', '$f$']
data = [X1, X2, X]

fig = plt.figure(figsize=(16, 9), dpi=400)
for n, title, d in zip(range(131, 134), titles, data):
    plt.subplot(n, facecolor='white', title=title)
    plt.pcolor(xgrid, tgrid, d.real)
plt.colorbar()
plt.show()

In [ ]:
from numpy.linalg import pinv

EPSILON = 1E-8

n = len(x)
A = X[1:].T @ pinv(X[:-1].T, rcond=EPSILON)
assert A.shape == (n, n)
assert np.allclose(A @ X[:-1].T, X[1:].T, rtol=EPSILON, atol=EPSILON)

In [ ]:
from numpy.linalg import eig, svd

class DynamicModeDecomposition:
    def __init__(self, rank):
        self.rank = rank

    def fit(self, x: np.ndarray):
        if not isinstance(x, np.ndarray):
            raise TypeError()
        if len(x.shape) != 2 or len(x) >= x.shape[1] or len(x) < 2:
            raise ValueError()
        x, x_prime = x[:-1].T, x[1:].T
        u, s, vh = svd(x, full_matrices=False)
        assert np.allclose(x, u * s @ vh, rtol=EPSILON, atol=EPSILON)
        u, s, vh = u[:, :self.rank], s[:self.rank], vh[:self.rank]
        assert x.shape == (u * s @ vh).shape
        atilde = np.conj(u.T) @ x_prime @ np.conj(vh.T) / s
        w, v = eig(atilde)
        assert np.allclose(atilde @ v, w * v, rtol=EPSILON, atol=EPSILON)
        phi = x_prime @ np.conj(vh.T) / s @ v
        return self

    def predict(self, x: np.array):
        return x

In [ ]:
from pydmd import DMD

dmd = DMD(svd_rank=2).fit(X.T)
for i in range(1, len(t)):
    assert np.allclose(dmd.predict(X[i - 1]), X[i], rtol=1E-8, atol=1E-8)

In [ ]:
DynamicModeDecomposition(2).fit(X)